<a href="https://colab.research.google.com/github/tort-cam/ST-554-P1/blob/main/Task1/task01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Title: Task 1

Author: Bryan Sandor

# Initialization

The data being made available through the UCI website, a package must be installed before the appropriate file may be accessed and the necessary variables saved locally.

In [ ]:
!pip install ucimlrepo

import pandas as pd
import math as math

from ucimlrepo import fetch_ucirepo

# fetch dataset
air_quality = fetch_ucirepo(id=360)

# data (as pandas dataframes)
airdata = air_quality.data.features
y = air_quality.data.targets

# metadata
print(air_quality.metadata)

# variable information
print(air_quality.variables)

# look at a few of the observations
airdata

{'uci_id': 360, 'name': 'Air Quality', 'repository_url': 'https://archive.ics.uci.edu/dataset/360/air+quality', 'data_url': 'https://archive.ics.uci.edu/static/public/360/data.csv', 'abstract': 'Contains the responses of a gas multisensor device deployed on the field in an Italian city. Hourly responses averages are recorded along with gas concentrations references from a certified analyzer. ', 'area': 'Computer Science', 'tasks': ['Regression'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 9358, 'num_features': 15, 'feature_types': ['Real'], 'demographics': [], 'target_col': None, 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Sun Mar 10 2024', 'dataset_doi': '10.24432/C59K5F', 'creators': ['Saverio Vito'], 'intro_paper': {'ID': 420, 'type': 'NATIVE', 'title': 'On field calibration of an electronic nose for benzene estimation in an urban pollution monitoring scenario', 'authors': 

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


Now we delete any observations where `C6H6(GT)` or `PT08.S1(CO)` are $-200$, which represent missing values.

In [ ]:
sum(airdata["C6H6(GT)"] == -200) # count number of observations to delete

366

In [ ]:
for i in range(len(airdata)):
    if (airdata["C6H6(GT)"].loc[i] == -200):
        airdata.drop(i, inplace = True) # inplace updates dataframe actively

In [ ]:
sum(airdata["C6H6(GT)"] == -200)

0

Notice the observations indicating missing values for `C6H6(GT)` must have also been missing values for `PT08.S1(CO)` since the number of observations indicating missing values for the latter is now also $0$. A `len` command also verifies $366$ observations have been removed from the dataframe.

In [ ]:
sum(airdata["PT08.S1(CO)"] == -200)

0

In [ ]:
len(airdata)

8991

# Loss Function

In [133]:
mesh = (airdata["C6H6(GT)"].quantile(0.75) - airdata["C6H6(GT)"].quantile(0.25))/1000
mesh

grid = [x * mesh + airdata["C6H6(GT)"].quantile(0.25) for x in range(1000)]

In [118]:
def loss(y = "C6H6(GT)", c = 0):

    """
    This function takes a column from the airdata data frame (by default the C6H6(GT) column) and a fixed value c (by default 0) and returns the root mean square error of all the observations in the desired column and c.
    """

    MSE = 0 # initialize mean square error (MSE)

    for i in range(len(airdata)):
        MSE += ( airdata[y].iloc[i] - c ) ** 2 # calculate MSE

    RMSE = math.sqrt(MSE) # calculate root MSE (RMSE)

    return RMSE

In [163]:
rmsegrid = [[grid[i], loss(y = "C6H6(GT)", c = grid[i])] for i in range(1000)]

In [164]:
rmsegrid

[[np.float64(4.4), 888.4434816013913],
 [np.float64(4.4096), 887.8916544548433],
 [np.float64(4.4192), 887.3404179469328],
 [np.float64(4.428800000000001), 886.789773179096],
 [np.float64(4.438400000000001), 886.2397212543372],
 [np.float64(4.448), 885.690263277188],
 [np.float64(4.4576), 885.1414003537245],
 [np.float64(4.4672), 884.5931335916249],
 [np.float64(4.476800000000001), 884.0454641000256],
 [np.float64(4.486400000000001), 883.498392989689],
 [np.float64(4.496), 882.9519213728406],
 [np.float64(4.5056), 882.4060503632998],
 [np.float64(4.5152), 881.860781076388],
 [np.float64(4.5248), 881.3161146289326],
 [np.float64(4.534400000000001), 880.7720521393494],
 [np.float64(4.5440000000000005), 880.2285947275255],
 [np.float64(4.5536), 879.6857435148972],
 [np.float64(4.5632), 879.143499624406],
 [np.float64(4.5728), 878.6018641805034],
 [np.float64(4.582400000000001), 878.0608383091511],
 [np.float64(4.5920000000000005), 877.5204231378302],
 [np.float64(4.6016), 876.98061979554]

TypeError: list indices must be integers or slices, not tuple

In [177]:
min(rmsegrid[])

TypeError: list indices must be integers or slices, not tuple

In [169]:
airdatac6mean = airdata["C6H6(GT)"].mean()

In [168]:
loss(c = airdatac6mean)

706.3592030258148